In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import sklearn

In [2]:
#Code to load the pickled data

def loadData(x): 
    # for reading also binary mode is important 
    dbfile = open(x, 'rb')      
    db = pickle.load(dbfile) 
    #for keys in db: 
     #   print(keys, '=>', db[keys]) 
    dbfile.close()
    return db

In [3]:
pear_corr = loadData('Pearson_Correlations')
user_data = pd.read_pickle("User_data.pkl")


In [4]:
user_data

,UserID,Ratings,Date,MovieID,rating_mean,Adjusted_Ratings
0,915,5,2005-08-1,1,4.285714,0.714286
1,1333,4,2004-05-1,3,2.676000,1.324000
2,967,1,2005-01-2,4,3.625000,-2.625000
3,685,3,2005-11-3,5,4.133333,-1.133333
4,1333,3,2005-04-0,8,2.676000,0.324000
5,695,1,2005-10-1,8,3.320988,-2.320988
6,1856,5,2005-06-2,8,4.425532,0.574468
7,1983,3,2005-04-1,8,3.740741,-0.740741
8,7,5,2005-07-3,8,4.066667,0.933333
9,307,3,2005-09-2,8,3.666667,-0.666667


In [15]:
unique_movies = user_data["MovieID"].unique()

In [21]:
#Lets consider the top K = 30 users that are similar to the user in question
K = 30

#here, userA = userID of the required user
#pear_corr = correlation dictionary between two users
#user_data = Ratings Matrix

from collections import Counter

def predict_rating(userA, K, pear_corr, user_data, unique_movies):
    final_lst = []
    for movieZ in unique_movies:
        print(movieZ)
        dict_target = {}
        for key in pear_corr:
            user_derived = int(key.split("_")[0])
            target = int(key.split("_")[-1])
            if user_derived == userA:
                dict_target[target] = pear_corr[str(userA) + "_" + str(target)]
        #print(dict_target)
        sorted_desc = Counter(dict_target)
        sorted_desc = sorted_desc.most_common()
        #here, sorted_desc is a list with (userID, correlation pair)
        sorted_desc = sorted_desc[:K]
        #print(sorted_desc)
    
        mean_rating_A = user_data[user_data["UserID"] == userA]["rating_mean"].iloc[0]
    
        final_bool = False
    
        for userB in sorted_desc:
            #print(userB)
            val = ((user_data["UserID"] == userB[0]) & (user_data["MovieID"] == movieZ)).any()
            final_bool = final_bool | val
        
        if final_bool == False:
            continue
    
        add_val = 0.0
        divisor = 0.0
    
        for userB in sorted_desc:
            val = ((user_data["UserID"] == userB[0]) & (user_data["MovieID"] == movieZ)).any()
            if val == True:
                adj_ratB = user_data.loc[(user_data["UserID"] == userB[0]) & (user_data["MovieID"] == movieZ)]['Adjusted_Ratings'].iloc[0]
                add_val += adj_ratB * userB[-1]
                divisor += userB[-1]
    
        final_lst.append([movieZ, mean_rating_A + (add_val/divisor)])
                         
    return final_lst

In [27]:
rat = predict_rating(1871, K, pear_corr, user_data, unique_movies)

1
3
4
5
8
17
18
24
26
28
30
33
36
44
45
46
48
52
55
56
57
58
70
71
76
77
78
79
83
84
85
88
95
96
97
101
104
106
107
108
110
111
117
118
122
126
127
128
129
132
138
143
148
152
155
156
157
158
164
165
166
167
169
171
173
175
178
180
181
185
187
188
189
191
197
199
201
206
208
209
210
211
213
215
216
222
223
225
227
231
232
238
240
241
248
252
253
254
255
256
257
262
263
268
269
270
271
273
275
280
281
283
285
288
289
290
294
295
297
299
302
305
311
312
313
314
316
321
322
325
327
329
330
331
333
334
337
340
344
345
348
353
356
357
358
359
360
361
362
363
367
368
371
375
376
378
381
385
386
390
393
395
397
398
401
403
405
406
408
409
410
413
414
415
416
420
421
422
424
425
426
428
429
430
431
433
436
442
443
445
451
452
455
457
458
459
462
463
467
468
471
472
473
474
475
476
477
482
483
488
490
494
497
499
500
501
504
505
509
511
513
516
517
518
520
524
527
528
531
533
534
535
548
550
551
557
560
561
563
564
566
569
570
571
575
577
580
582
585
586
587
588
589
593
595
599
607
610
613
615


4082
4086
4088
4089
4091
4092
4094
4095
4097
4098
4099
4100
4105
4106
4109
4115
4117
4118
4119
4123
4125
4127
4131
4134
4135
4136
4138
4141
4144
4145
4147
4148
4149
4150
4154
4155
4157
4159
4161
4162
4164
4165
4167
4168
4171
4173
4174
4175
4177
4179
4181
4185
4189
4193
4197
4198
4201
4207
4210
4211
4214
4216
4218
4219
4225
4227
4234
4237
4238
4240
4244
4247
4248
4249
4251
4252
4254
4255
4256
4257
4259
4260
4262
4263
4266
4267
4268
4269
4271
4273
4281
4282
4284
4287
4289
4290
4291
4293
4296
4298
4299
4302
4306
4310
4315
4320
4322
4327
4330
4331
4334
4335
4336
4337
4339
4340
4341
4342
4345
4348
4352
4353
4354
4355
4356
4360
4361
4364
4366
4369
4370
4371
4372
4374
4377
4378
4379
4380
4381
4383
4384
4385
4386
4389
4390
4392
4393
4394
4396
4398
4402
4405
4406
4409
4411
4414
4415
4418
4420
4421
4422
4427
4429
4432
4433
4439
4441
4442
4450
4452
4453
4454
4460
4465
4472
4474
4478
4479
4480
4483
4485
4488
4490
4492
4493
4496
4497
4499


In [28]:
print(rat)

[[8, 2.801742919389978], [17, 3.1601307189542482], [18, 2.8346647300895667], [28, 3.522923584957886], [30, 3.816313696060718], [46, 2.8324244885330856], [52, 3.331216054496916], [77, 1.8346647300895667], [78, 4.213536291709667], [106, 4.331216054496916], [111, 3.8343637818684484], [143, 3.8347984773321313], [148, 2.7194164291350997], [171, 4.834664730089567], [175, 3.857977138413957], [187, 2.279785811539975], [191, 3.6541888365419517], [197, 5.1535947712418295], [199, 5.16320659721303], [208, 2.8324244885330856], [225, 4.160130718954248], [241, 4.331216054496916], [252, -27.94891849689661], [255, 0.6879382055852643], [256, 4.834664730089567], [257, 4.619699135198081], [268, 4.834664730089567], [269, 3.1700563562143427], [270, 4.038554432463228], [273, 1.910890735349598], [275, 2.2135362917096666], [289, 3.8324244885330856], [290, 2.640592127571731], [295, 3.4308107390261235], [299, 3.4193568559192187], [311, 3.8346647300895667], [312, 3.999845360894898], [313, 3.6192288944001882], [32

In [29]:
rat_sorted = sorted(rat, key=lambda x: x[1], reverse=True)

In [31]:
print(rat_sorted)

[[2680, 10.366441337639746], [3197, 8.398849901262105], [1289, 7.2479588951073115], [2290, 5.853103493976556], [672, 5.566152299183973], [763, 5.566152299183973], [1528, 5.566152299183973], [2443, 5.566152299183973], [4256, 5.566152299183973], [2130, 5.331216054496916], [3078, 5.21047154540781], [199, 5.16320659721303], [4429, 5.160130718954248], [197, 5.1535947712418295], [886, 5.134407441746472], [2942, 5.09509843149723], [2177, 5.0538472964943555], [933, 4.936714429361488], [1370, 4.936714429361488], [1629, 4.936714429361488], [3513, 4.936714429361488], [1255, 4.888198823980304], [2953, 4.8814598852923705], [171, 4.834664730089567], [256, 4.834664730089567], [268, 4.834664730089567], [443, 4.834664730089567], [624, 4.834664730089567], [673, 4.834664730089567], [872, 4.834664730089567], [954, 4.834664730089567], [964, 4.834664730089567], [1094, 4.834664730089567], [1175, 4.834664730089567], [1595, 4.834664730089567], [1627, 4.834664730089567], [1700, 4.834664730089567], [1741, 4.8346